In [ ]:
**The Battle of Neighbourhoods: IBM Capstone Project**

In [ ]:
**Introduction** 
As part of the course we looked at location data for Toronto and New York, these cities had a lot of replies from the Foursquare API calls so I decided to look at smaller cities that are expanding and test if
a similar comparison can be done with them, the aim is to contribute to future interested parties for example data analysts, Tourists, location data managers, if there are limitations that might need to be resolved using a different API 
or collecting more data to feed into the requests etc.

In this project the comparison is between Toronto and London Ontario as I wanted to have a city which I had to start compiling the data as there was not much location data that was already compiled but also a city near Toronto 
so that can also inform anyone wanting to explore further than Toronto example Tourists on other places of interest.

Toronto is a multicultural city in the Province of Ontario, it is the most populated city in Canada, it had a population of over 2.7 million in 2016 (Canadian census),also has the highest proportion of minority ethnic population in Canada. 
Its economy is diversified: Art, Design, Banking & Finance, Life Science, Technology, Aerospace, Fashion, Education, Tourism etc, thus an attractive location for many headquarters of multinational corporations,
it welcomes many immigrants annually thus attracting an inflow of talent and opportunities for innovation and expansion as well as having influence on the neighbourhood demographics and tastes.
    
As urban centres grow some people move out to the suburbs to seek other job opportunities or choose to retire elsewhere, others move to be closer to friends and family, others seek a change etc. 
Where to relocate to depends on many factors e.g. job opportunities in their industry, average wages, housing, public/community provisions, crime rates, access to amenities, warmer weather etc.  

London is a city in Western Ontario which had a population of 383,822 in 2016 according to the Canadian census, it is about 120 miles from Toronto and Detroit. Strong industries are heathcare with several hospitals located in the city, 
Education e.g the University of Western Ontario (UWO), Insurance and Technology industries. for heathcare and education with several hospitals located. Also they hold cultural events, Tourists can also explore the settlement changes that 
happened after the natural disasters and immigration influence over time since the early settlers example links to the UK with many similar names and systems.   

  
I want to compare Downtown Toronto neighbourhoods and London Ontario using location data from Foursquare API, using the location data to look at the different neighbourhoods in these cities to find the most popular venues in each.


In [ ]:
**Data: The Foursquare API will be used to gather data on the different venues to analyse these urban neighbourhoods in Toronto and Western Ontario.
     Data scrapping and cleaning using python packages.
     Postcodes, neighbourhood, and Borough data for Toronto has been taken from a Wikipedia page List of postal codes of Canada: M - Wikipedia
     The data for Western Ontario has been taken from another Wikipedia page List of postal codes of Canada: N - Wikipedia

    Longitude and latitude data for Toronto was provided in week 3 of the course http://cocl.us/Geospatial_data
    Western Ontario geographical coordinates data was gathered from GeoNames

    I will cluster the neighbourhoods and choose top 10 most popular venues in each and plot them on a map using Folium to help visualise and interpret the data.


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import json

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm

import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium

print("Libraries Imported")

     |████████████████████████████████| 94 kB 5.8 MB/s  eta 0:00:01
